In [ ]:
import warnings
warnings.filterwarnings('ignore')
from tqdm.autonotebook import tqdm
#importing the libraries
import numpy as np
import pandas as pd
from pathlib import Path
import datetime
import pickle

path_dir = Path('D:/Workspace/Projects/BayesianLearning/Project1_Data/CSV')
path="D:/Workspace/Projects/BayesianLearning/Project1_Data/CSV/"
list_dfs = []
for path_file in path_dir.glob('Node_*.csv'):
    df_small = pd.read_csv(path_file)
    list_dfs.append(df_small)
    
df = pd.concat(list_dfs, axis=0)
df.reset_index(drop=True, inplace=True)

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%d/%m/00%y %H:%M')

In [ ]:
spatialT=df.sort_values(by=['timestamp'], ascending=True)
spatialT.reset_index(drop=True, inplace=True)
spatialH=df.sort_values(by=['timestamp'], ascending=True)
spatialH.reset_index(drop=True, inplace=True)
start = datetime.datetime(2014, 12, 1)

In [ ]:
spatialT.drop(["temperature","humidity"], axis=1, inplace=True)
spatialH.drop(["temperature","humidity"], axis=1, inplace=True)
spatialT['timestamp']=spatialT['timestamp']-start
spatialT['timestamp']=spatialT['timestamp'].apply(lambda x: x/np.timedelta64(1, 'm')).fillna(0).astype('int64')
spatialH['timestamp']=spatialH['timestamp']-start
spatialH['timestamp']=spatialH['timestamp'].apply(lambda x: x/np.timedelta64(1, 'm')).fillna(0).astype('int64')

In [ ]:
#Prediction Data Aggregator
ID2Index = {}
for k in tqdm([1,2,5,7,8]):
    check=pd.read_csv(path+'Node_50'+str(k)+'.csv')
    check['timestamp'] = pd.to_datetime(check['timestamp'], format='%d/%m/00%y %H:%M')
    check['timestamp'] = check['timestamp']-start
    check['timestamp'] = check['timestamp'].apply(lambda x: x/np.timedelta64(1, 'm')).fillna(0).astype('int64')
    spatialT['X'+str(k)] = np.nan
    spatialH['X'+str(k)] = np.nan
    for i in tqdm(range(spatialT.shape[0]), leave=False):
        if spatialT.iloc[i]['ID'] not in ID2Index:
            ID2Index[spatialT.iloc[i]['ID']]=i
        for j in range(check.shape[0]):
            if abs(spatialT.iloc[i]['timestamp']-check.iloc[j]['timestamp'])<=5:
                spatialT.at[i,'X'+str(k)] = check.iloc[j]['temperature']
                spatialH.at[i,'X'+str(k)] = check.iloc[j]['humidity']
                break

In [ ]:
spatialT.to_csv(path+'Sessions/spatialT.csv',index=False)
spatialH.to_csv(path+'Sessions/spatialH.csv',index=False)
with open(path+'Sessions/ID2Index', 'wb') as handle:
    pickle.dump(ID2Index, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
path_dir = Path('D:/Workspace/Projects/BayesianLearning/Project1_Data/CSV/Sessions')

list_dfs = []
for path_file in path_dir.glob('TN_50*.csv'):
    df_small = pd.read_csv(path_file)
    list_dfs.append(df_small)
    
df = pd.concat(list_dfs, axis=0)
df.reset_index(drop=True, inplace=True)

In [ ]:
testT=df.sort_values(by=['timestamp'], ascending=True)
testT.reset_index(drop=True, inplace=True)
testH=df.sort_values(by=['timestamp'], ascending=True)
testH.reset_index(drop=True, inplace=True)
testT['timestamp'] = pd.to_datetime(testT['timestamp'], format='%Y-%m-%d %H:%M:%S')
testH['timestamp'] = pd.to_datetime(testH['timestamp'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
testT.drop(["temperature","humidity"], axis=1, inplace=True)
testH.drop(["temperature","humidity"], axis=1, inplace=True)

In [ ]:
testT['timestamp']=testT['timestamp']-start
testT['timestamp']=testT['timestamp'].apply(lambda x: x/np.timedelta64(1, 'm')).fillna(0).astype('int64')
testH['timestamp']=testH['timestamp']-start
testH['timestamp']=testH['timestamp'].apply(lambda x: x/np.timedelta64(1, 'm')).fillna(0).astype('int64')

In [ ]:
#Test Data Aggregator
TestID2Index = {}
for k in tqdm([1,2,5,7,8]):
    check=pd.read_csv(path+'Sessions/TN_50'+str(k)+'.csv')
    check['timestamp'] = pd.to_datetime(check['timestamp'], format='%Y-%m-%d %H:%M:%S')
    check['timestamp'] = check['timestamp']-start
    check['timestamp'] = check['timestamp'].apply(lambda x: x/np.timedelta64(1, 'm')).fillna(0).astype('int64')
    testT['X'+str(k)] = np.nan
    testH['X'+str(k)] = np.nan
    for i in tqdm(range(testT.shape[0]), leave=False):
        if testT.iloc[i]['ID'] not in TestID2Index:
            TestID2Index[testT.iloc[i]['ID']]=i
        for j in range(check.shape[0]):
            if abs(testT.iloc[i]['timestamp']-check.iloc[j]['timestamp'])<=5:
                testT.at[i,'X'+str(k)] = check.iloc[j]['temperature']
                testH.at[i,'X'+str(k)] = check.iloc[j]['humidity']
                break

In [ ]:
testT.to_csv(path+'Sessions/testT.csv',index=False)
testH.to_csv(path+'Sessions/testH.csv',index=False)
with open(path+'Sessions/TestID2Index', 'wb') as handle:
    pickle.dump(TestID2Index, handle, protocol=pickle.HIGHEST_PROTOCOL)